LSE Data Science Institute | ME204 (2023/24) | Week 01 Day 03

# 🗓️ Week 01 – Day 03: Summarizing and Visualizing Data

(MORNING NOTEBOOK)

**DATE:** 10 July 2024

**AUTHOR:** Dr [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

-----


## ⚙️ Setup

- Ensure the Python kernel has the necessary libraries: `pandas`, `matplotlib` and `lets-plot`.
- Ensure the `bakery.csv` file is in the `data` folder.

**Imports**

(It is a good practice to import ALL the libraries you will be using at the start of your notebook)

In [1]:
import numpy as np
import pandas as pd

from lets_plot import *
LetsPlot.setup_html()

# 1. Reading & tidying up the data a bit

In [8]:
filename = '../data/bakery.csv' # Can you change this to the file you want to read?
df = pd.read_csv(filename)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   data-product-id        485 non-null    int64 
 1   data-product-name      485 non-null    object
 2   data-product-type      485 non-null    object
 3   data-product-on-offer  485 non-null    bool  
 4   data-product-index     485 non-null    int64 
 5   image-url              485 non-null    object
 6   product-page           485 non-null    object
 7   product-name           485 non-null    object
 8   product-size           482 non-null    object
 9   item-price             485 non-null    object
 10  price-per-unit         463 non-null    object
 11  offer-description      52 non-null     object
 12  category               485 non-null    object
dtypes: bool(1), int64(2), object(10)
memory usage: 46.1+ KB


Dropping unnecessary columns and renaming the columns for better understanding:

In [9]:
# Drop duplicates
df = df.drop_duplicates()

df = df.drop(columns=['data-product-name', 
                      'data-product-type', 
                      'data-product-index', 
                      'category'])
df = (
    df.rename(columns={
        'data-product-id': 'id',
        'data-product-price': 'price',
        'data-product-on-offer': 'offer',
        'product-page': 'page',
        'product-name': 'name',
        'product-size': 'size',
    })
)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 482 entries, 0 to 484
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 482 non-null    int64 
 1   offer              482 non-null    bool  
 2   image-url          482 non-null    object
 3   page               482 non-null    object
 4   name               482 non-null    object
 5   size               479 non-null    object
 6   item-price         482 non-null    object
 7   price-per-unit     460 non-null    object
 8   offer-description  49 non-null     object
dtypes: bool(1), int64(1), object(7)
memory usage: 34.4+ KB


Changing types of columns:

In [10]:
# The id does not need 64 bits. 32 bits is enough.
# See https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.intc for ranges.
df['id'] = df['id'].astype('int32')

## 1.1 Fixing the `item-price` column

The `item-price` column is a string, it has things like `£3.00` and `60p`. Before we can convert it to a number, we need to remove the `£` and `p` symbols and convert it to a number.


Follow my Live Demo as I build the rationale for the solution you see below. 

In [21]:
df.shape

(482, 9)

**Building up the intuition of our data cleaning routine with `for` and `if-else`:**

In [26]:
item_price_list = df['item-price'].tolist()

new_list = []

for element in item_price_list:
    if '£' in element:
       new_element = element.replace('£', '')
    elif 'p' in element:
       new_element = element.replace('p', '')
       new_element = '0.' + new_element
    else:
       new_element = element

    new_list.append(new_element)

df['item-price-fixed'] = new_list
df['item-price-fixed'] = df['item-price-fixed'].astype(float)

**Convert the routine above to a function, to demonstrate how to use `apply()`:**

In [36]:
def clean_item_price(item_price: str):
    if '£' in item_price:
        clean_item_price = item_price.replace('£', '')
    elif 'p' in item_price:
        clean_item_price = item_price.replace('p', '')
        clean_item_price = '0.' + clean_item_price
    else:
        clean_item_price = item_price

    return clean_item_price

Test that the function works as intended:

In [41]:
new_list = []
for item_price in item_price_list:
    new_list.append(clean_item_price(item_price))

This could be simplified with a **list comprehension**:

In [42]:
new_list = [clean_item_price(item_price) for item_price in item_price_list]

**Use pandas `.apply()` for that:**

In [48]:
df['item-price-fixed'] = df['item-price'].apply(clean_item_price).astype(float)

If I only cared about dropping the `£` symbol, then I could write a simple **lambda function** instead (a one-liner function without a name):

In [51]:
df['item-price'].apply(lambda item_price: item_price.replace('£', ''))

0       1.98
1       2.36
2       2.96
3       1.31
4       1.90
       ...  
480     2.25
481    30.00
482     2.60
483     3.00
484     2.10
Name: item-price, Length: 482, dtype: object

In [50]:
df[['item-price', 'item-price-fixed']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 482 entries, 0 to 484
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item-price        482 non-null    object 
 1   item-price-fixed  482 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.3+ KB


In [52]:
df.loc[df['item-price'].str.contains('p'), 'item-price'] = df['item-price'].apply(lambda x: '0.' + str.replace(x, 'p', ''))
df.loc[df['item-price'].str.contains('£'), 'item-price'] = df['item-price'].str.replace('£', '')
df['item-price'] = df['item-price'].astype('float')

In [55]:
df = df.drop(columns=['item-price-fixed'])

# 2. EDA by means of curiosity-driven questions

## Q1: What is the distribution of prices in the Waitrose Bakery section?

In [60]:
df['item-price'].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
item-price,482.0,4.84668,7.750208,0.5,1.6,2.2,3.15,45.0


We could visualize some key statistics about the `item-price` on a histogram:

In [67]:
(
    ggplot(data=df, mapping=aes(x="item-price")) +
    geom_histogram() + 
    geom_vline(xintercept=4.84668, color="red") +
    geom_vline(xintercept=2.2, color="blue")
)

Let's say I'm interested in understanding the price of 🍞 bread products:

## Q2: How many bread products are there in the dataset?

In [74]:
sample_string = "bread A"

'bread' in sample_string or 'Bread' in sample_string

True

In [79]:
df['name'].apply(lambda product_name: 'bread' in product_name or 'Bread' in product_name).sum()

52

In [80]:
all_bread = df['name'].str.contains('bread', case=False)

print(f"There are {all_bread.sum()} bread products in the dataset.")

There are 52 bread products in the dataset.


## Q3: Are they all truly bread? Or do I have some other products with the string `'bread'` in the name?

In [90]:
# Follow my live demo to understand the process of writing the code below.
df[all_bread][['name', 'size', 'item-price', 'page']].sort_values(by=['name', 'size']).set_index(['page'])

,name,size,item-price
page,,,
https://www.waitrose.com/ecom/products/all-butter-shortbread/056597-28405-28406,All Butter Shortbread,each,1.20
https://www.waitrose.com/ecom/products/bfree-gluten-free-wholegrain-pitta-breads/657631-695118-695119,BFree Gluten Free Wholegrain Pitta Breads,4x55g,2.90
https://www.waitrose.com/ecom/products/bacheldr-rustic-crunch-bread-mix/559129-220498-220499,Bacheldr Rustic Crunch Bread Mix,500g,1.50
https://www.waitrose.com/ecom/products/cohens-bakery-buckingham-rye-bread/077133-39207-39208,Cohens Bakery Buckingham Rye Bread,400g,2.10
https://www.waitrose.com/ecom/products/crosta-mollica-piadina-flatbreads/817933-198092-198093,Crosta & Mollica Piadina Flatbreads,3s,2.00
https://www.waitrose.com/ecom/products/crosta-mollica-wholeblend-flatbreads/807541-307196-307197,Crosta & Mollica Wholeblend Flatbreads,4s,2.00
https://www.waitrose.com/ecom/products/deli-kitchen-greek-style-flatbreads/831341-702286-702287,Deli Kitchen Greek Style Flatbreads,4s,1.90
https://www.waitrose.com/ecom/products/deli-kitchen-plain-folded-flatbreads/521824-492967-492968,Deli Kitchen Plain Folded Flatbreads,6s,1.50
https://www.waitrose.com/ecom/products/deli-kitchen-seeded-folded-flatbreads/884168-493355-493356,Deli Kitchen Seeded Folded Flatbreads,6s,1.50


🧑‍⚖️ **DECISION:** 

- Remove 'shortbread'
- Remove 'flatbread'
- Remove 'bread mix'
- Remove 'pitta bread'

Pure Python vs Pandas logical operators:

- `and` -> `&`
- `or` -> `|`
- `not` -> `~`

In [102]:
all_valid_breads = (
    df['name'].str.contains('bread', case=False) & 
    ~df['name'].str.contains('gingerbread', case=False) &
    ~df['name'].str.contains('flatbread', case=False) &
    ~df['name'].str.contains('pitta bread', case=False) &
    ~df['name'].str.contains('bread mix', case=False) &
    ~df['name'].str.contains('shortbread', case=False)
)

# df[all_valid_breads]

In [103]:
breads_to_remove = ['shortbread', 'flatbread', 'bread mix', 'pitta bread', 'gingerbread']

# Rebuild all_breads to exclude the breads_to_remove
all_bread = (
    df['name'].str.contains('bread', case=False) & 
    ~df['name'].str.contains('|'.join(breads_to_remove), case=False)
)

print(f"There are {all_bread.sum()} bread products in the dataset.")

There are 31 bread products in the dataset.


In [104]:
df_bread = df[all_bread].sort_values(['name', 'size'])

df_bread[['name', 'size', 'item-price', 'page']].set_index(['page'])

,name,size,item-price
page,,,
https://www.waitrose.com/ecom/products/cohens-bakery-buckingham-rye-bread/077133-39207-39208,Cohens Bakery Buckingham Rye Bread,400g,2.10
https://www.waitrose.com/ecom/products/essential-white-medium-sliced-bread/055018-27631-27632,Essential White Medium Sliced Bread,800g,0.75
https://www.waitrose.com/ecom/products/essential-wholemeal-medium-sliced-bread/055051-27670-27671,Essential Wholemeal Medium Sliced Bread,800g,0.75
https://www.waitrose.com/ecom/products/hovis-1886-granary-sliced-bread/841477-746623-746624,Hovis 1886 Granary Sliced Bread,450g,1.50
https://www.waitrose.com/ecom/products/hovis-1886-seeded-sliced-bread/531774-746615-746616,Hovis 1886 Seeded Sliced Bread,450g,1.50
https://www.waitrose.com/ecom/products/hovis-best-of-both-medium-sliced-bread/084740-42991-42992,Hovis Best of Both Medium Sliced Bread,800g,1.35
https://www.waitrose.com/ecom/products/hovis-granary-wholemeal-sliced-bread/054266-27239-27240,Hovis Granary Wholemeal Sliced Bread,800g,1.90
https://www.waitrose.com/ecom/products/hovis-original-granary-medium-sliced-bread/055416-27817-27818,Hovis Original Granary Medium Sliced Bread,800g,1.90
https://www.waitrose.com/ecom/products/hovis-original-granary-thick-sliced-bread/055312-27767-27768,Hovis Original Granary Thick Sliced Bread,400g,1.25


## Q4: Which sizes are available for each bread product?

⭐️ GET READY FOR YOUR FIRST `groupby()`!

- Follow my live demo closely as I explain the difference between `pd.Series` and `pd.DataFrame` as they explain the output of the code below. 
- You will also learn about the `apply()` method (not shown here yet)

In [108]:
# This is the simpler solution, but why does it look odd and different to the data we've been seeing in previous steps?
df_bread.groupby('name')['size'].unique()

name
Cohens Bakery Buckingham Rye Bread                      [400g]
Essential White Medium Sliced Bread                     [800g]
Essential Wholemeal Medium Sliced Bread                 [800g]
Hovis 1886 Granary Sliced Bread                         [450g]
Hovis 1886 Seeded Sliced Bread                          [450g]
Hovis Best of Both Medium Sliced Bread                  [800g]
Hovis Granary Wholemeal Sliced Bread                    [800g]
Hovis Original Granary Medium Sliced Bread              [800g]
Hovis Original Granary Thick Sliced Bread               [400g]
Hovis Seed Sensations Multiseeded Sliced Bread    [400g, 800g]
Hovis Soft White Medium Sliced White Bread              [800g]
Hovis Soft White Thick Sliced White Bread               [800g]
Hovis Wholemeal Medium Sliced Bread               [400g, 800g]
Hovis Wholemeal Thick Sliced Bread                      [800g]
Irwin's Together Brown Soda Bread                       [400g]
Livlife Seriously Seeded Sliced Bread             

## Q5: How many sizes are available for each bread product?

What if I want a count, not the sizes themselves?

In [109]:
df_bread.groupby('name')['size'].nunique()

name
Cohens Bakery Buckingham Rye Bread                1
Essential White Medium Sliced Bread               1
Essential Wholemeal Medium Sliced Bread           1
Hovis 1886 Granary Sliced Bread                   1
Hovis 1886 Seeded Sliced Bread                    1
Hovis Best of Both Medium Sliced Bread            1
Hovis Granary Wholemeal Sliced Bread              1
Hovis Original Granary Medium Sliced Bread        1
Hovis Original Granary Thick Sliced Bread         1
Hovis Seed Sensations Multiseeded Sliced Bread    2
Hovis Soft White Medium Sliced White Bread        1
Hovis Soft White Thick Sliced White Bread         1
Hovis Wholemeal Medium Sliced Bread               2
Hovis Wholemeal Thick Sliced Bread                1
Irwin's Together Brown Soda Bread                 1
Livlife Seriously Seeded Sliced Bread             1
No.1 Malt Sourdough Bread with Seeds              1
No.1 Rye and Wheat Dark Sourdough Bread           1
No.1 Spelt Sourdough Bread                        1
No.1 Wh

**🎯 ACTION POINTS:**

- Create a solution where you have two columns: `available_sizes` and `num_sizes`
- Your final data frame should look like this:

<div style="margin-left:3em">

| name | available_sizes | num_sizes |
|------|-----------------|-----------|

</div>

- Sort the resulting DataFrame by `num_sizes` in descending order

<div style="margin-left:2em;padding-left:1em;font-size:0.75em;width:60%">

💡 **HINT:** To create a new column in pandas, use the following syntax:

```python
df['new_column'] = df['old_column'].apply(lambda x: x + 1)
```

</div>

In [118]:
# A merge solution
a = df_bread.groupby('name')['size'].unique()
b = df_bread.groupby('name')['size'].nunique()

tmp_df = pd.merge(a, b, on=['name'])
tmp_df.columns = ['available_sizes', 'num_sizes']
tmp_df

,available_sizes,num_sizes
name,,
Cohens Bakery Buckingham Rye Bread,[400g],1
Essential White Medium Sliced Bread,[800g],1
Essential Wholemeal Medium Sliced Bread,[800g],1
Hovis 1886 Granary Sliced Bread,[450g],1
Hovis 1886 Seeded Sliced Bread,[450g],1
Hovis Best of Both Medium Sliced Bread,[800g],1
Hovis Granary Wholemeal Sliced Bread,[800g],1
Hovis Original Granary Medium Sliced Bread,[800g],1
Hovis Original Granary Thick Sliced Bread,[400g],1


In [124]:
tmp_df = a.to_frame("available_sizes")
tmp_df['num_sizes'] = b
tmp_df

,available_sizes,num_sizes
name,,
Cohens Bakery Buckingham Rye Bread,[400g],1
Essential White Medium Sliced Bread,[800g],1
Essential Wholemeal Medium Sliced Bread,[800g],1
Hovis 1886 Granary Sliced Bread,[450g],1
Hovis 1886 Seeded Sliced Bread,[450g],1
Hovis Best of Both Medium Sliced Bread,[800g],1
Hovis Granary Wholemeal Sliced Bread,[800g],1
Hovis Original Granary Medium Sliced Bread,[800g],1
Hovis Original Granary Thick Sliced Bread,[400g],1


More function-based approach:

In [133]:
def get_size_summary(rows):
    result = {"num_sizes": len(rows['size']),
              "available_sizes": set(rows['size'])}
    return pd.Series(result)


df_bread.groupby(['name']).apply(get_size_summary)

,num_sizes,available_sizes
name,,
Cohens Bakery Buckingham Rye Bread,1,{400g}
Essential White Medium Sliced Bread,1,{800g}
Essential Wholemeal Medium Sliced Bread,1,{800g}
Hovis 1886 Granary Sliced Bread,1,{450g}
Hovis 1886 Seeded Sliced Bread,1,{450g}
Hovis Best of Both Medium Sliced Bread,1,{800g}
Hovis Granary Wholemeal Sliced Bread,1,{800g}
Hovis Original Granary Medium Sliced Bread,1,{800g}
Hovis Original Granary Thick Sliced Bread,1,{400g}


## Q6: How different are the prices of sliced vs unsliced breads?

**🎯 ACTION POINTS:**

Now let's do something more complex! 

<span style="display:block;margin-left:1.5em;font-size:0.85em;">If you manage to solve this, then you would have already built the skills to solve the 💻 [Week 01 Day 01 Lab](https://lse-dsi.github.io/ME204/2024/weeks/week01/day01/lab.html) - 🎁 [Bonus Tasks (Challenge)](https://lse-dsi.github.io/ME204/2024/weeks/week01/day01/lab.html#bonus-tasks)!</span>

<div style="display:inline-flex;flex-wrap:wrap;flex-direction:row;width:80%;margin-left:0.5em">

<div style="width:400px;height:260px;border-radius:1em;margin:1%;padding:1.5%;background-color:#fafafa">

<h2>Is it sliced?</h2>

Create a new column on the DataFrame of breads called `is_sliced` and fill it with `True` if the product is sliced and `False` otherwise.

</div>

<div style="width:400px;height:260px;border-radius:1em;margin:1%;padding:1.5%;background-color:#fafafa">

<h2>Price per kg</h2>

Create a new column on the DataFrame of breads called `price-per-kg`. 

Check if `price-per-unit` is suitable. If not, replace it with the price per 100g of the product.

</div>
<!-- 
<div style="width:400px;height:460px;border-radius:1em;margin:1%;padding:1.5%;background-color:#fafafa">

<h2>Variety</h2>

Create a new column on the DataFrame of breads called `variety` and fill it with the variety of the product. 

For example, the variety of 

> "Hovis Wholemeal Medium Sliced Bread" 

is "Wholemeal Medium Sliced Bread" 

And the variety of 

> "Irwin's Together Malted Grain Bread" 

is "Malted Grain Bread"

</div>

<div style="width:400px;height:460px;border-radius:1em;margin:1%;padding:1.5%;background-color:#fafafa">

<h2>Brand</h2>

Create a new column on the DataFrame of breads and call it `brand`.  Fill it with the brand of the product. 

For example, the brand of 

> Hovis Wholemeal Medium Sliced Bread

is "Hovis"

And the brand of 

> Irwin's Together Malted Grain Bread 

is "Irwin's Together"

</div> -->

</div>


**Then, compare the distribution of prices of sliced vs unsliced breads.**

In [111]:
# WRITE YOUR CODE HERE

(
    df_bread.assign(is_sliced=lambda x: x['name'].str.contains('sliced', case=False),
                    price_per_kg=lambda x: x['item-price'] / x['size'].str.replace('g', '').astype('float') * 1000)
            .rename(columns={'price_per_kg': 'price-per-kg'})
            .groupby('is_sliced')
            .apply(lambda x: x['price-per-kg'].describe())
)

price-per-kg,count,mean,std,min,25%,50%,75%,max
is_sliced,,,,,,,,
False,13.0,4.921795,1.417702,3.0000,4.4000,5.000,5.40000,7.300000
True,18.0,2.647832,1.209149,0.9375,1.8125,2.375,3.28125,5.090909


## Want to practice some more EDA?

Why stop on bread? 🍞

Take another look at the original list of products under `df` and see if you can find other interesting questions to ask and answer. Try comparing the prices of different products, or the number of sizes available for each product. The sky is the limit! 

# ☕️ Time for a Coffee Break!

# 3. Visualizing the data 

We will use the [lets-plot](https://lets-plot.org/) library to create some visualizations. `lets-plot` is a Python implementation of the popular `ggplot2` library in R, which is the most powerful example of the concept of the **Grammar of Graphics**.

In order to get the visualisation to work, I first need to rework the data a bit. To achieve the same result as me, you must find a way to create a DataFrame with the exact content as the one below.

<div style="width:80%;font-size:0.65em;margin-left:1em">

| name                                           | price   | size   |   price-per-kg | is_sliced   | offer   | offer_description   |
|:-----------------------------------------------|:--------|:-------|---------------:|:------------|:--------|:--------------------|
| Cohens Bakery Buckingham Rye Bread             | £ 2.10  | 400g   |        5.25    | Unsliced    | False   | Not On Offer        |
| Essential White Medium Sliced Bread            | £ 0.75  | 800g   |        0.9375  | Sliced      | False   | Not On Offer        |
| Essential Wholemeal Medium Sliced Bread        | £ 0.75  | 800g   |        0.9375  | Sliced      | False   | Not On Offer        |
| Hovis 1886 Granary Sliced Bread                | £ 1.50  | 450g   |        3.33333 | Sliced      | True    | save 30p. Was £1.80 |
| Hovis 1886 Seeded Sliced Bread                 | £ 1.50  | 450g   |        3.33333 | Sliced      | True    | save 30p. Was £1.80 |
| Hovis Best of Both Medium Sliced Bread         | £ 1.35  | 800g   |        1.6875  | Sliced      | False   | Not On Offer        |
| Hovis Granary Wholemeal Sliced Bread           | £ 1.90  | 800g   |        2.375   | Sliced      | False   | Not On Offer        |
| Hovis Original Granary Medium Sliced Bread     | £ 1.90  | 800g   |        2.375   | Sliced      | False   | Not On Offer        |
| Hovis Original Granary Thick Sliced Bread      | £ 1.25  | 400g   |        3.125   | Sliced      | False   | Not On Offer        |
| Hovis Seed Sensations Multiseeded Sliced Bread | £ 1.20  | 400g   |        3       | Sliced      | False   | Not On Offer        |
| Hovis Seed Sensations Multiseeded Sliced Bread | £ 1.90  | 800g   |        2.375   | Sliced      | False   | Not On Offer        |
| Hovis Soft White Medium Sliced White Bread     | £ 1.45  | 800g   |        1.8125  | Sliced      | False   | Not On Offer        |
| Hovis Soft White Thick Sliced White Bread      | £ 1.45  | 800g   |        1.8125  | Sliced      | False   | Not On Offer        |
| Hovis Wholemeal Medium Sliced Bread            | £ 1.10  | 400g   |        2.75    | Sliced      | False   | Not On Offer        |
| Hovis Wholemeal Medium Sliced Bread            | £ 1.45  | 800g   |        1.8125  | Sliced      | False   | Not On Offer        |
| Hovis Wholemeal Thick Sliced Bread             | £ 1.45  | 800g   |        1.8125  | Sliced      | False   | Not On Offer        |
| Irwin's Together Brown Soda Bread              | £ 2.00  | 400g   |        5       | Unsliced    | False   | Not On Offer        |
| Livlife Seriously Seeded Sliced Bread          | £ 2.00  | 500g   |        4       | Sliced      | False   | Not On Offer        |
| No.1 Malt Sourdough Bread with Seeds           | £ 2.20  | 500g   |        4.4     | Unsliced    | False   | Not On Offer        |
| No.1 Rye and Wheat Dark Sourdough Bread        | £ 2.70  | 500g   |        5.4     | Unsliced    | False   | Not On Offer        |
| No.1 Spelt Sourdough Bread                     | £ 2.70  | 500g   |        5.4     | Unsliced    | False   | Not On Offer        |
| No.1 White Sourdough Bread                     | £ 2.20  | 500g   |        4.4     | Unsliced    | False   | Not On Offer        |
| Schneider Brot Rye Bread with Sunflower Seeds  | £ 1.50  | 500g   |        3       | Unsliced    | False   | Not On Offer        |
| Seeded Sourdough Bread                         | £ 2.25  | 500g   |        4.5     | Unsliced    | False   | Not On Offer        |
| The Heart of Nature Pure Grain Bread           | £ 3.65  | 500g   |        7.3     | Unsliced    | False   | Not On Offer        |
| Vogel's Original Mixed Grain Bread             | £ 2.40  | 800g   |        3       | Unsliced    | False   | Not On Offer        |
| Vogel's Soya & Linseed Bread                   | £ 2.40  | 800g   |        3       | Unsliced    | False   | Not On Offer        |
| Wildfarmed Seeded Sourdough Bread              | £ 4.00  | 600g   |        6.66667 | Unsliced    | False   | Not On Offer        |
| Wildfarmed Sliced Seeded Bread                 | £ 2.80  | 550g   |        5.09091 | Sliced      | False   | Not On Offer        |
| Wildfarmed Sliced White Bread                  | £ 2.80  | 550g   |        5.09091 | Sliced      | False   | Not On Offer        |
| Wildfarmed White Sourdough Bread               | £ 4.00  | 600g   |        6.66667 | Unsliced    | False   | Not On Offer        |

</div>

Here's what you get when you run the `info()` method on this DataFrame:

<div style="width:30%;font-size:0.75em;margin-left:1em">

```python
<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 484 to 244
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               31 non-null     object 
 1   price              31 non-null     object 
 2   size               31 non-null     object 
 3   price-per-kg       31 non-null     float64
 4   is_sliced          31 non-null     object 
 5   offer              31 non-null     bool   
 6   offer_description  31 non-null     object 
dtypes: bool(1), float64(1), object(5)
memory usage: 1.7+ KB
```

</div>

💡 TIP: **Pay close attention as I explain the concept and the code below!**

If we don't have time to create the dataframe above together, I'll share the solution later.

<div style="color:transparent;background-color:transparent">

<details style="height:0.1em"><summary></summary>
plot_df = (
    df_bread.assign(is_sliced=lambda x: x['name'].str.contains('sliced', case=False),
                    price_per_kg=lambda x: x['item-price'] / x['size'].str.replace('g', '').astype('float') * 1000,
                    price=lambda x: x['item-price'].apply(lambda x: f"£ {x:.2f}"),
                    offer_description=lambda x: x['offer-description'].apply(lambda d: d if type(d) == str else 'Not On Offer'))
            .assign(is_sliced=lambda x: x['is_sliced'].map({True: 'Sliced', False: 'Unsliced'}))
            .drop(columns=['id', 'item-price', 'image-url', 'page', 'price-per-unit', 'offer-description'])
            .rename(columns={'price_per_kg': 'price-per-kg'})
            [['name', 'price', 'size', 'price-per-kg', 'is_sliced', 'offer', 'offer_description']]
)

print(plot_df.to_markdown(index=False))
</details>
</div>

In [113]:
plot_df = (
    df_bread.assign(is_sliced=lambda x: x['name'].str.contains('sliced', case=False),
                    price_per_kg=lambda x: x['item-price'] / x['size'].str.replace('g', '').astype('float') * 1000,
                    price=lambda x: x['item-price'].apply(lambda x: f"£ {x:.2f}"),
                    offer_description=lambda x: x['offer-description'].apply(lambda d: d if type(d) == str else 'Not On Offer'))
            .assign(is_sliced=lambda x: x['is_sliced'].map({True: 'Sliced', False: 'Unsliced'}))
            .drop(columns=['id', 'item-price', 'image-url', 'page', 'price-per-unit', 'offer-description'])
            .rename(columns={'price_per_kg': 'price-per-kg'})
            [['name', 'price', 'size', 'price-per-kg', 'is_sliced', 'offer', 'offer_description']]
)

plot_df

,name,price,size,price-per-kg,is_sliced,offer,offer_description
484,Cohens Bakery Buckingham Rye Bread,£ 2.10,400g,5.250000,Unsliced,False,Not On Offer
178,Essential White Medium Sliced Bread,£ 0.75,800g,0.937500,Sliced,False,Not On Offer
79,Essential Wholemeal Medium Sliced Bread,£ 0.75,800g,0.937500,Sliced,False,Not On Offer
227,Hovis 1886 Granary Sliced Bread,£ 1.50,450g,3.333333,Sliced,True,save 30p. Was £1.80
61,Hovis 1886 Seeded Sliced Bread,£ 1.50,450g,3.333333,Sliced,True,save 30p. Was £1.80
29,Hovis Best of Both Medium Sliced Bread,£ 1.35,800g,1.687500,Sliced,False,Not On Offer
104,Hovis Granary Wholemeal Sliced Bread,£ 1.90,800g,2.375000,Sliced,False,Not On Offer
56,Hovis Original Granary Medium Sliced Bread,£ 1.90,800g,2.375000,Sliced,False,Not On Offer
54,Hovis Original Granary Thick Sliced Bread,£ 1.25,400g,3.125000,Sliced,False,Not On Offer
23,Hovis Seed Sensations Multiseeded Sliced Bread,£ 1.20,400g,3.000000,Sliced,False,Not On Offer


In [114]:
(
    ggplot(data=plot_df, 
           mapping=aes(x='is_sliced', y='price-per-kg', fill='is_sliced')) +
    geom_jitter(width=0.15, height=0, alpha=0.75, size=5, stroke=1.2, color="black", shape=21,
                tooltips=layer_tooltips().line('@name').line('@size').line('@price').line('@offer_description')) +
    geom_boxplot(width=0.35, alpha=0.35, color='black') +
    scale_x_discrete(name='') +
    scale_y_continuous(name='Price per kg (£)', breaks=list(range(0, 10)), limits=[0, 8], format='£ {.2f}') +
    labs(title='Sliced bread is consistently cheaper!', 
         subtitle='A comparison of the price per kg of sliced and unsliced bread',
         caption='Hover your mouse over the points to see the details') +

     theme(axis_text_x=element_text(size=17),
           axis_text_y=element_text(size=17),
           axis_title_x=element_text(size=20),
           axis_title_y=element_text(size=20),
           plot_title=element_text(size=22, face='bold'),
           plot_subtitle=element_text(size=18),
           legend_position='none') +
     ggsize(700, 400)
)

----

# Now what?

- **Practice!** You will receive a new dataset this afternoon to practice some more EDA and we'll practice these same concepts tomorrow.
- **Bookmark the [lets-plot documentation](https://lets-plot.org/)**. Official documentation is the best place to use as a reference.